## 0. Install and Import Dependencies

In [ ]:
!pip install ibm_watson

In [ ]:
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

## 1. Setup STT Service

In [ ]:
apikey = ''
url = ''

In [ ]:
authenticator = IAMAuthenticator(apikey)
stt = SpeechToTextV1(authenticator = authenticator)
stt.set_service_url(url)

## 2. Compress and Split Audio

In [ ]:
import subprocess 
import os
command = 'ffmpeg -i audio.wav -vn -ar 44100 -ac 2 -b:a 192k audio.mp3'
subprocess.call(command, shell=True)
command = 'ffmpeg -i audio.mp3 -f segment -segment_time 360 -c copy %03d.mp3'
subprocess.call(command, shell=True)

In [ ]:
files = []
for filename in os.listdir('.'):
    if filename.endswith(".mp3") and filename !='audio.mp3':
        files.append(filename)
files.sort()

In [ ]:
files

## 3. Perform Conversion

In [ ]:
results = []
for filename in files:
    with open(filename, 'rb') as f:
        res = stt.recognize(audio=f, content_type='audio/mp3', model='en-AU_NarrowbandModel', continuous=True, \
                           inactivity_timeout=360).get_result()
        results.append(res)

In [ ]:
text = []
for file in results:
    for result in file['results']:
        text.append(result['alternatives'][0]['transcript'].rstrip() + '.\n')

In [ ]:
text

In [ ]:
with open('output.txt', 'w') as out:
    out.writelines(text)